# 🎥 Colab Video Compressor

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AdittyaMondal/ColabVideoCompressor/blob/main/colab_notebook.ipynb)

Advanced video compression bot with GPU acceleration and multiple presets.

In [ ]:
#@title 🎥 Advanced Video Compressor Bot { display-mode: "form" }
#@markdown ### Configure and Run Your Video Compression Bot

import os
from IPython.display import HTML, display
from google.colab import files

#@markdown ## 1. Basic Configuration
APP_ID = "Your_APP_ID"  #@param {type:"string"}
API_HASH = "Your_API_HASH"  #@param {type:"string"}
BOT_TOKEN = "Your_BOT_TOKEN" #@param {type:"string"}
OWNER = "Your_Telegram_User_ID" #@param {type:"string"}
THUMBNAIL = "https://envs.sh/F82.jpg, Your_Thumbnail_URL" #@param {type:"string"}

#@markdown ## 2. Select Compression Preset
preset_option = "Balanced (Recommended)" #@param ["Ultra Fast (Lowest Quality)", "Balanced (Recommended)", "High Quality", "Max Quality (Slow)", "Custom"]

# Define presets dictionary with fixed formatting
FFMPEG_PRESETS = {
    "Ultra Fast (Lowest Quality)": 'ffmpeg -i "{}" -preset ultrafast -c:v libx265 -crf 35 -vf scale=-2:720 -threads 16 -tune fastdecode -x265-params pools=16:frame-threads=16:no-wpp=1:no-pmode=1:no-pme=1:ref=1:min-keyint=1:keyint=24:bframes=0 -map 0:v -c:a aac -b:a 128k -map 0:a -c:s copy -map 0:s? "{}"',

    "Balanced (Recommended)": 'ffmpeg -i "{}" -preset veryfast -c:v libx265 -crf 28 -vf scale=-2:720 -threads 16 -tune fastdecode -x265-params pools=16:frame-threads=16:ref=3:bframes=3:aq-mode=2:psy-rd=1.0:psy-rdoq=2.0 -map 0:v -c:a aac -b:a 192k -map 0:a -c:s copy -map 0:s? "{}"',

    "High Quality": 'ffmpeg -i "{}" -preset slow -c:v libx265 -crf 23 -vf scale=-2:1080:flags=lanczos -threads 16 -x265-params pools=16:frame-threads=16:ref=5:bframes=5:aq-mode=3:psy-rd=1.0:psy-rdoq=2.0:deblock=1,1 -map 0:v -c:a aac -b:a 256k -map 0:a -c:s copy -map 0:s? "{}"',

    "Max Quality (Slow)": 'ffmpeg -i "{}" -preset veryslow -c:v libx265 -crf 18 -vf scale=-2:1080:flags=lanczos -threads 16 -x265-params pools=16:frame-threads=16:ref=6:bframes=6:aq-mode=3:psy-rd=1.0:psy-rdoq=2.0:deblock=1,1:rc-lookahead=60 -map 0:v -c:a aac -b:a 320k -map 0:a -c:s copy -map 0:s? "{}"'
}

#@markdown ### 3. Custom Preset Options (if selected above)
if preset_option == "Custom":
    #@markdown #### Video Settings
    resolution = "720p" #@param ["480p", "720p", "1080p"]
    crf_value = 28 #@param {type:"slider", min:18, max:35, step:1}
    preset_speed = "veryfast" #@param ["ultrafast", "superfast", "veryfast", "faster", "fast", "medium", "slow", "slower", "veryslow"]
    video_bitrate = "2000k" #@param ["1000k", "2000k", "3000k", "4000k", "5000k", "6000k"]

    #@markdown #### Audio Settings
    audio_bitrate = "128k" #@param ["128k", "192k", "256k", "320k"]

    custom_preset = f'ffmpeg -i "{{}}" -preset {preset_speed} -c:v libx265 -b:v {video_bitrate} -crf {crf_value} -vf scale=-2:{resolution[:-1]} -threads 16 -x265-params pools=16:frame-threads=16:ref=4:bframes=4:aq-mode=2 -map 0:v -c:a aac -b:a {audio_bitrate} -map 0:a -c:s copy -map 0:s? "{{}}"'

    FFMPEG_PRESETS["Custom"] = custom_preset

# Setup function with improved error handling
def setup_bot():
    print("🚀 Starting bot setup...")

    # Validate inputs
    if not BOT_TOKEN or not OWNER:
        raise ValueError("❌ BOT_TOKEN and OWNER ID are required!")

    # Install required packages
    !apt-get -qq update
    !apt-get -qq install -y ffmpeg mediainfo

    print("📦 Packages installed successfully")

    # Clone and setup repository
    !git clone https://github.com/AdittyaMondal/ColabVideoCompressor.git
    %cd ColabVideoCompressor

    # Create directories
    !mkdir -p encode downloads
    !chmod 777 encode downloads

    print("📁 Directories created successfully")

    # Create .env file with proper formatting
    env_content = f"""APP_ID={APP_ID}
    API_HASH={API_HASH}
    BOT_TOKEN={BOT_TOKEN}
    OWNER={OWNER}
    THUMBNAIL={THUMBNAIL}
    FFMPEG={FFMPEG_PRESETS[preset_option]}"""

    with open('.env', 'w') as f:
        f.write(env_content)

    !chmod 644 .env

    print("⚙️ Environment configured successfully")

    # Install Python dependencies
    !pip install -r requirements.txt

    print("🤖 Starting bot...")
    # Start the bot
    !python -m bot

#@markdown ## 4. Run Setup and Start Bot
run_setup = True #@param {type:"boolean"}

if run_setup:
    try:
        if not BOT_TOKEN or not OWNER:
            print("❌ Error: Please provide both BOT_TOKEN and OWNER ID before running!")
        else:
            setup_bot()
    except Exception as e:
        print(f"❌ Error occurred: {str(e)}")

# Display helpful information
display(HTML('''
<div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px; margin-top: 20px;">
<h3>📋 Important Notes:</h3>
<ul>
    <li><strong>BOT_TOKEN:</strong> Get from @BotFather on Telegram</li>
    <li><strong>OWNER ID:</strong> Get from @userinfobot (just send /start to it)</li>
    <li><strong>Ultra Fast Preset:</strong> Best for quick 10-minute compressions</li>
    <li><strong>Connection Issues?</strong> Try running the cell again</li>
</ul>
<p><em>For best results under 10 minutes, use the "Ultra Fast" preset with 720p resolution.</em></p>
</div>
'''))